<a href="https://colab.research.google.com/github/arimbawa/Big-Data/blob/main/06_b_Analisis_Data_Relasional_Menggunakan_Neo4j_(Graph_Database).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Analisis Data Relasional Menggunakan Neo4j (Graph Database)

**Tujuan:**

1. Mahasiswa memahami konsep dasar Graph Database dan property graph model.
2. Mahasiswa mampu membuat node, relationship, dan query menggunakan Cypher Query Language.
3. Mahasiswa mengenal penggunaan Neo4j untuk analisis hubungan (misalnya jejaring sosial atau kolaborasi penelitian).

**Apa itu Neo4j?**

1. Neo4j adalah database berbasis graf, bukan tabel.
2. Data direpresentasikan sebagai:
3. Node → entitas (orang, institusi, artikel, produk)
4. Relationship → hubungan antar node
5. Property → atribut tambahan (misalnya nama, tahun, skor)

Contoh struktur:</p>
(Agus:Person {name:'Agus'})-[:COLLABORATES_WITH]->(Budi:Person {name:'Budi'})

# Langkah 1: Setup di Google Colab
Jalankan ini terlebih dahulu di Colab:

In [2]:
!pip install neo4j

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.8/325.8 kB 14.1 MB/s eta 0:00:00


# Langkah 2: Siapkan Database Neo4j (AuraDB Free)

1. Buka: https://neo4j.com/product/auradb/
2. Login menggunakan akun Google.
3. Buat database baru.
4. Setelah aktif, catat detail koneksi seperti:
  - Connection URI: neo4j+s://xxxxx.databases.neo4j.io
  - Username: neo4j
  - Password: (dibuat saat setup)

# Langkah 3: Koneksi dari Colab ke Neo4j

In [4]:
# Ganti dengan kredensial Anda
uri = "neo4j+s://XXXXX.databases.neo4j.io"
username = "XXXX"
password = "XXXXXXXXXX"

from neo4j import GraphDatabase

driver = GraphDatabase.driver(uri, auth=(username, password))

print("Koneksi ke Neo4j berhasil!")

Koneksi ke Neo4j berhasil!


# Langkah 4: Membuat Node dan Relationship



In [5]:
def create_data(tx):
    # Hapus data lama
    tx.run("MATCH (n) DETACH DELETE n")

    # Buat node Person dan Relationship
    tx.run("""
        CREATE (a:Person {name:'Agus', bidang:'IoT'}),
               (b:Person {name:'Budi', bidang:'AI'}),
               (c:Person {name:'Citra', bidang:'Data Science'}),
               (d:Person {name:'Dewi', bidang:'IoT'}),
               (a)-[:COLLABORATES_WITH]->(b),
               (b)-[:COLLABORATES_WITH]->(c),
               (a)-[:COLLABORATES_WITH]->(d)
    """)

with driver.session() as session:
    session.execute_write(create_data)

print("Data graf berhasil dibuat!")

Data graf berhasil dibuat!


# Langkah 5: Menjalankan Query Cypher

Menampilkan seluruh node:

In [6]:
with driver.session() as session:
    result = session.run("MATCH (n) RETURN n.name AS nama, n.bidang AS bidang")
    for record in result:
        print(record["nama"], "-", record["bidang"])

Agus - IoT
Budi - AI
Citra - Data Science
Dewi - IoT


Menampilkan relasi kolaborasi:

In [7]:
with driver.session() as session:
    result = session.run("""
        MATCH (a)-[r:COLLABORATES_WITH]->(b)
        RETURN a.name AS from, b.name AS to
    """)
    for record in result:
        print(f"{record['from']} → {record['to']}")

Agus → Budi
Agus → Dewi
Budi → Citra


# Langkah 6: Analisis Sederhana
a. Temukan siapa yang berkolaborasi dengan “Agus”:

In [8]:
query = """
MATCH (a:Person {name:'Agus'})-[:COLLABORATES_WITH]->(partner)
RETURN partner.name AS kolaborator
"""
with driver.session() as session:
    result = session.run(query)
    for record in result:
        print("Kolaborator Agus:", record["kolaborator"])

Kolaborator Agus: Budi
Kolaborator Agus: Dewi


b. Temukan semua path kolaborasi dari Agus ke Citra:

In [9]:
query = """
MATCH path = (a:Person {name:'Agus'})-[*]-(b:Person {name:'Citra'})
RETURN path
"""
with driver.session() as session:
    result = session.run(query)
    print("Path ditemukan:", len(list(result)) > 0)

Path ditemukan: True


# Diskusi

1. Apa keuntungan menggunakan Neo4j dibanding SQL untuk data hubungan?
2. Bagaimana model graf ini cocok untuk analisis jejaring sosial, kolaborasi paten, atau riset akademik?
3. Bagaimana integrasi Neo4j dalam sistem Big Data (misalnya layer analisis relasi pada data lake)?